## Classical Decomposition Model

$$
    X_t = m_t + s_t + Y_t, \ \ 1 \leq t \leq n
$$
where $E[Y_t] = 0$, $s_t = s_{t + d}$ and $\sum_{i = 1}^{d} s_i = 0$. 


In [28]:
def f(w):
    def g(x):
        return (x * w).sum() / (len(w) - 1)
    return g

def calculate_initial_trend(x, d):
    assert d > 0
    
    if d % 2 == 0:
        wts = [0.5] + [1] * (d - 1) + [0.5]
        return x.rolling(window = d + 1, center = True).apply(f(wts))
    
    return x.rolling(window = d, center = True).mean()

def estimate_seasonal_component(x, d):
    ini_trend = calculate_initial_trend(x, d)
    

In [29]:
import pandas as pd

df = pd.DataFrame(columns=['x'])
df['x'] = [i for i in range(1, 11)]

In [30]:
df

,x
0,1
1,2
2,3
3,4
4,5
5,6
6,7
7,8
8,9
9,10


In [31]:
trend = calculate_initial_trend(df['x'], d=4)

/home/somnath/Envs/env3.6/lib/python3.6/site-packages/ipykernel_launcher.py:11: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  # This is added back by InteractiveShellApp.init_path()


In [32]:
trend

0    NaN
1    NaN
2    3.0
3    4.0
4    5.0
5    6.0
6    7.0
7    8.0
8    NaN
9    NaN
Name: x, dtype: float64